# Chatea con imágenes de páginas PDF

**Si estás buscando la aplicación web, revisa la carpeta src/.** 

Este notebook demuestra cómo convertir páginas de PDF a imágenes y enviarlas a un modelo de visión para inferencia

## Autentícate en OpenAI

El siguiente código se conecta a OpenAI, ya sea usando una cuenta de Azure OpenAI, modelos de GitHub, o modelo local de Ollama. Mira el README para instrucciones sobre cómo configurar el archivo `.env`.

In [ ]:
import os

import azure.identity
import openai
from dotenv import load_dotenv

load_dotenv(".env", override=True)

openai_host = os.getenv("OPENAI_HOST", "github")

if openai_host == "github":
    print("Usando GitHub Models con GITHUB_TOKEN como clave")
    openai_client = openai.OpenAI(
        api_key=os.environ["GITHUB_TOKEN"],
        base_url="https://models.github.ai/inference",
    )
    model_name = os.getenv("OPENAI_MODEL", "openai/gpt-4o")
elif openai_host == "local":
    print("Usando API local compatible con OpenAI sin clave")
    openai_client = openai.OpenAI(api_key="no-key-required", base_url=os.environ["LOCAL_OPENAI_ENDPOINT"])
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")
elif openai_host == "azure" and os.getenv("AZURE_OPENAI_KEY_FOR_CHATVISION"):
    # Autenticación usando una clave de API de Azure OpenAI
    # Esto generalmente no se recomienda, pero se proporciona por conveniencia
    print("Usando Azure OpenAI con clave")
    openai_client = openai.OpenAI(
        base_url=os.environ["AZURE_OPENAI_ENDPOINT"] + "/openai/v1/",
        api_key=os.environ["AZURE_OPENAI_KEY_FOR_CHATVISION"],
    )
    # Esto es en realidad el nombre del deployment, no el nombre del modelo
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")
elif openai_host == "azure" and os.getenv("AZURE_OPENAI_ENDPOINT"):
    tenant_id = os.environ["AZURE_TENANT_ID"]
    print("Usando Azure OpenAI con credencial de Azure Developer CLI para tenant id", tenant_id)
    default_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=tenant_id)
    token_provider = azure.identity.get_bearer_token_provider(
        default_credential, "https://cognitiveservices.azure.com/.default"
    )
    openai_client = openai.OpenAI(
        base_url=os.environ["AZURE_OPENAI_ENDPOINT"] + "/openai/v1/",
        api_key=token_provider,
    )
    # Esto es en realidad el nombre del deployment, no el nombre del modelo
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")

print(f"Usando modelo {model_name}")

## Convierte PDFs a imágenes

In [ ]:
%pip install Pillow PyMuPDF

In [ ]:
import pymupdf
from PIL import Image

filename = "../plants.pdf"
doc = pymupdf.open(filename)
for i in range(doc.page_count):
    page = doc.load_page(i)
    pix = page.get_pixmap()
    original_img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    original_img.save(f"page_{i}.png")
doc.close()

## Envía imágenes al modelo de visión

In [ ]:
import base64


def open_image_as_base64(filename):
    with open(filename, "rb") as image_file:
        image_data = image_file.read()
    image_base64 = base64.b64encode(image_data).decode("utf-8")
    return f"data:image/png;base64,{image_base64}"

In [ ]:
user_content = [{"text": "¿Qué plantas están listadas en estas páginas?", "type": "text"}]
# Procesa solo las primeras páginas, ya que procesar todas las páginas (doc.page_count) es lento
for i in range(3):
    user_content.append({"image_url": {"url": open_image_as_base64(f"../page_{i}.png")}, "type": "image_url"})

response = openai_client.chat.completions.create(model=model_name, messages=[{"role": "user", "content": user_content}])

print(response.choices[0].message.content)